In [1]:
IMAGE_DIM=64

from dataset import ForexGafData
import torch
import os
import pandas as pd

df = pd.read_csv('./data/eurusd_minute.csv')
dataset = ForexGafData(df['BidClose'], IMAGE_DIM)

for data in dataset:
    assert not torch.isnan(data).any()

print('n:', len(dataset))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('using', device)

/home/jekabs/anaconda3/envs/ml_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


n: 80690
using cuda


In [3]:
from models import Generator, Discriminator
from trainer import WGanTrainer, WGanGpTrainer, ClassicalGanTrainer
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np


g = Generator()
g._initialize_weights()
d = Discriminator()

g_optim = torch.optim.Adam(g.parameters(), lr=1e-4, betas=(0.0, 0.9))
d_optim = torch.optim.Adam(d.parameters(), lr=1e-4, betas=(0.0, 0.9))

# g_optim = torch.optim.RMSprop(g.parameters(), lr=5e-5)
# d_optim = torch.optim.RMSprop(d.parameters(), lr=5e-5)

write_dir='./runs/last'
model_dir='./models/last'

if not os.path.isdir(write_dir):
    os.makedirs(write_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

trainer = WGanGpTrainer(
    generator=g,
    critic=d,
    gen_optimizer=g_optim,
    critic_optimizer=d_optim,
    latent_dimension=100,
    device=device,
    write_dir=write_dir,
    model_dir=model_dir,
    checkpoint_interval=1,
    gp_weight=10,
)


data_loader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)

# with torch.autograd.detect_anomaly():
trainer.train(data_loader, epochs=3)

Training...


Epoch [1/3]: 100%|██████████| 2521/2521 [04:02<00:00, 10.40it/s]


Saving model checkpoint...


Epoch [2/3]: 100%|██████████| 2521/2521 [04:00<00:00, 10.46it/s]


Saving model checkpoint...


Epoch [3/3]:  12%|█▏        | 311/2521 [00:29<03:31, 10.45it/s]


KeyboardInterrupt: 